In [1]:
import numpy as np
import math

# You might find the functions in this notebook helpful for your implementation.

## Printing

In [2]:
def print_sudoku(board_2d_int_arr):
    X = board_2d_int_arr
    r,c = X.shape
    a = int(math.sqrt(r))
    assert r == c
    assert type(X) == np.ndarray
    assert a**2 == r 
    assert isinstance(X[0][0].item(),int)
    
    # Convert array elements to strings
    board_str = X.astype(str)
    
    # Our row separator
    row_sep = '-'*25

    # Loop through 9 rows
    for i in range(r):
        
        # At each multiple of 3, print row separator
        if i % a == 0:
            print(row_sep)

        # Get row data
        row = board_str[i]

        # Format row of data with pipe separators at each end, and between each sub grid
        print('| '+' '.join(row[0:a])+' | '+' '.join(row[a:2*a])+' | '+' '.join(row[2*a:])+' |')

    # Print final row separator at bottom after loops finish
    print(row_sep)

In [3]:
board = np.zeros(shape=(9,9),dtype=int)
board[0,:] = [1,9,2,4,6,8,5,7,3]
board[1,:] = [8,5,3,1,2,7,4,9,6] 
board[2,:] = [7,6,4,9,5,3,8,1,2]
board[3,:] = [5,8,9,7,3,6,1,2,4]
board[4,:] = [4,7,1,8,9,2,3,6,5]
board[5,:] = [2,3,6,5,4,1,7,8,9]
board[6,:] = [9,1,5,6,7,4,2,3,8]
board[7,:] = [3,4,7,2,8,9,6,5,1]
board[8,:] = [6,2,8,3,1,5,9,4,7]

In [4]:
print_sudoku(board)

-------------------------
| 1 9 2 | 4 6 8 | 5 7 3 |
| 8 5 3 | 1 2 7 | 4 9 6 |
| 7 6 4 | 9 5 3 | 8 1 2 |
-------------------------
| 5 8 9 | 7 3 6 | 1 2 4 |
| 4 7 1 | 8 9 2 | 3 6 5 |
| 2 3 6 | 5 4 1 | 7 8 9 |
-------------------------
| 9 1 5 | 6 7 4 | 2 3 8 |
| 3 4 7 | 2 8 9 | 6 5 1 |
| 6 2 8 | 3 1 5 | 9 4 7 |
-------------------------


## Check if the current board is a valid sudoku

In [5]:
def block_to_coordinate(id,k):
    # Takes in the block number id, and the kth element in that block,
    # Output the coordinate of that cell.
    assert(isinstance(id,int))
    assert(isinstance(k,int))

    a = id // 3
    b = id % 3

    i = 3*a + (k//3)
    j = 3*b + (k%3)

    return (i,j)


In [6]:
def check_board(input_board):
    """Check if the current board is a valid finished sudoku"""
    X = input_board
    r,c = X.shape
    n = r
    a = int(math.sqrt(r))
    assert r == c
    assert type(X) == np.ndarray
    assert a**2 == r
    assert isinstance(X[0][0].item(),int)


    flag = True
    for row in range(r):
        if len(np.unique(X[row])) != len(X[row]):
            flag = False
            # print("row is wrong:",row)

    for col in range(c):
        if len(np.unique(X[:,col])) != len(X[:,col]):
            flag = False
            # print("col is wrong:",col)
    
    for block_id in range(n):
        temp_arr = np.zeros(n,dtype=int)
        for k in range(n):
            r_idx, c_idx = block_to_coordinate(block_id,k)
            temp_arr[k] = X[r_idx][c_idx]
        if len(np.unique(temp_arr)) != len(temp_arr):
            flag = False
            # print("block is wrong:",block_id)

    return flag


In [7]:
check_board(board)

True

In [8]:
board2 = np.zeros(shape=(9,9),dtype=int)
board2[0,:] = [1,9,2,4,6,8,5,7,3]
board2[1,:] = [8,5,3,1,2,7,4,9,6] 
board2[2,:] = [7,6,4,9,5,3,8,1,2]
board2[3,:] = [5,8,9,7,3,6,1,2,4]
board2[4,:] = [4,7,1,8,9,2,3,6,5]
board2[5,:] = [2,3,6,5,4,1,7,8,9]
board2[6,:] = [9,1,5,6,7,4,2,3,8]
board2[7,:] = [3,4,7,2,8,9,7,5,1]
board2[8,:] = [6,2,8,3,1,5,9,4,7]
# Here we changed the 8th row, 7th column (indexed from 1 to 9) element from 6 to 7.

In [9]:
check_board(board2)

False

## Helper functions that relate to 3d probability vector representation of a sudoku board

In [10]:
def to_onehot(int_x,n=9):
    """ Convert integer x between 1 and 9 to its one-hot probablity vector encoding. """

    assert 1 <= int_x and int_x <= n
    onehot_vec = np.zeros(n)
    onehot_vec[int_x-1] = 1
    return onehot_vec

In [11]:
def convert_to_3d_repr(nonzero_board):
    """This function converts board with all NONZERO entries to 3d probability representation. """

    r,c = nonzero_board.shape
    n = r
    a = int(math.sqrt(r))
    assert r == c
    assert type(nonzero_board) == np.ndarray
    assert a**2 == r 

    prob_3d_arr = np.zeros(shape=(r,c,n))

    for i in range(r):
        for j in range(c):
            prob_3d_arr[i][j] = to_onehot(nonzero_board[i][j])

    return prob_3d_arr


In [12]:
def convert_to_board(prob_3d_arr):
    """This function converts a 3d probability vector representation back to the normal 2d board representation"""
    
    X = prob_3d_arr
    assert type(X) == np.ndarray
    r,c,n = X.shape
    assert r == c and c == n

    board = np.zeros(shape=(r,c), dtype=int)

    for i in range(r):
        for j in range(c):
            prob_vec = X[i][j]
            num = np.argmax(prob_vec)+1
            board[i][j] = num

    return board


### If you want to convert an initial board (with possibly many zero entries) to a 3d probability representation, you may find the following function helpful

In [13]:
def init_from_board(start_board):
    """Initialize probability 3d representation from a given board (2d).
    Given 'clue' cells are converted to one-hot encoding.
    Empty 'unkown' cells are converted to a uniformly random encoding over possible values, for example,
    an empty cell that can take values in {1,4,5,8} is converted to [0.25,0,0,0.25,0.25,0,0,0.25,0]
    
    """

    X = start_board
    r,c = X.shape
    n = r
    a = int(math.sqrt(r))
    assert r == c
    assert type(X) == np.ndarray
    assert a**2 == r 

    temp_3d_arr = np.ones(shape=(r,c,n))
    prob_3d_arr = np.ones(shape=(r,c,n))

    # First pass converts the nonzero entries to one-hot encodings, and
    # Set the correpsonding entries of temp_arr in same row/column/block to 0
    for i in range(r):
        for j in range(c):
            val = X[i][j]
            if val != 0:
                prob_3d_arr[i][j] = to_onehot(val)
                for k in range(n):
                    assert(isinstance(k,int))
                    temp_3d_arr[i][k][val-1] = 0  # set the entry in same row to have zero corresponding element
                    temp_3d_arr[k][j][val-1] = 0  # set the entry in same column to have zero corresponding element

                    # for same box
                    a = i // 3
                    b = j // 3
                    row_start_idx = 3*a
                    col_start_idx = 3*b

                    temp_3d_arr[row_start_idx+(k//3)][col_start_idx+(k%3)][val-1] = 0

    # Second pass normalizes the zero entries' corresponding vector to probability vector and assign it to prob_3d_arr
    for i in range(r):
        for j in range(c):
            val = X[i][j]
            if val == 0:
                prob_3d_arr[i][j] = temp_3d_arr[i][j]/np.sum(temp_3d_arr[i][j])

    return prob_3d_arr

